In [1]:
import xml.etree.ElementTree as ET
import os
import glob
import shutil

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import copy
import cv2
from PIL import Image, ImageDraw

In [2]:
# 원본이미지 폴더
basepath = '/home/ssac26/Downloads/wego/ai_test2'

In [3]:
os.path.dirname(basepath)

'/home/ssac26/Downloads/wego'

In [4]:
# txt변환후 이미지와 매칭되면 이동시킬 폴더
copy_path = os.path.join(os.path.dirname(basepath), os.path.basename(basepath)+'_copy')

In [5]:
copy_path
os.mkdir(copy_path)

In [6]:
bfolders = os.listdir(basepath)
bfolders

['Bbox_0892',
 'Bbox_0884',
 'Bbox_0863',
 'Bbox_0882',
 'Bbox_0864',
 'Bbox_0883',
 'Bbox_0887',
 'Bbox_0876',
 'Bbox_0869',
 'Bbox_0865',
 'Bbox_0886',
 'Bbox_0875',
 'Bbox_0870',
 'Bbox_0873',
 'Bbox_0895',
 'Bbox_0879',
 'Bbox_0893',
 'Bbox_0877',
 'Bbox_0894',
 'Bbox_0862',
 'Bbox_0867',
 'Bbox_0885',
 'Bbox_0878',
 'Bbox_0896',
 'Bbox_0891',
 'Bbox_0890',
 'Bbox_0889',
 'Bbox_0888',
 'Bbox_0871',
 'Bbox_0899',
 'Bbox_0874',
 'Bbox_0900',
 'Bbox_0897',
 'Bbox_0866',
 'Bbox_0868',
 'Bbox_0880',
 'Bbox_0881',
 'Bbox_0872',
 'Bbox_0861',
 'Bbox_0898']

In [7]:
# custom annotation format to yolov4
def convert(box, size):
    dw = 1./(size[0]) 
    dh = 1./(size[1])
    x = (box[0] + box[2])/2.0
    y = (box[1] + box[3])/2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)
    

In [8]:
def getimgdir(dir_path):
    image_list = []
    for i in glob.glob(dir_path + '/*.jpg'):
        image_list.append(i)
    
    return image_list

In [9]:
def gettextdir(dir_path):
    label_list = []
    for i in glob.glob(dir_path + '/*.txt'):
        label_list.append(i)
    
    return label_list

In [10]:
copy_path

'/home/ssac26/Downloads/wego/ai_test2_copy'

In [11]:
# parsing from xml, to txt
def parsing(path):
    new_folder = os.path.join(copy_path, os.path.basename(path))
    os.makedirs(new_folder) # make copy folder loc
    #print(new_folder)
    filelist = os.listdir(path)
    for file in filelist:

        if file.endswith('xml'):
            #print(file)
            xmlpath = os.path.join(path, file)
            shutil.copy(xmlpath, new_folder)
            doc = ET.parse(xmlpath)
            root = doc.getroot()
            for i in root.iter('image'):


               # cou = 0
                for j in i.iter('box'):


                    if j.attrib['label']=='person':

                        picname = i.attrib['name'].split('.')[0]



                        bbox_x1 = float(j.attrib['xtl'])
                        bbox_y1 = float(j.attrib['ytl'])
                        bbox_x2 = float(j.attrib['xbr'])
                        bbox_y2 = float(j.attrib['ybr'])
                        #print(convert([bbox_x1, bbox_y1, bbox_x2, bbox_y2],(1920,1080)))
                        xx, yy, xx2, yy2 = convert([bbox_x1, bbox_y1, bbox_x2, bbox_y2],(1920,1080))
                        #print(i.attrib['name'])
                        #print(i)
                        #print(j.attrib)
                        #print(xx, yy, xx2, yy2)
                        annotxt = os.path.join(path, picname) + '.txt'
                        #print(annotxt)
                        #print('osexist:',os.path.isfile(annotxt))
                        folder = os.listdir(path)

                        if os.path.basename(annotxt) in folder:

                            out_file = open(annotxt, 'a' ,encoding='UTF8')
                            #print()
                            out_file.write(f'{0} {round(xx,6)} {round(yy,6)} {round(xx2,6)} {round(yy2,6)}\n')
                            #print('addmode')
                        else:
                            out_file = open(os.path.join(path, picname) + '.txt', 'w' ,encoding='UTF8')

                            out_file.write(f'{0} {round(xx,6)} {round(yy,6)} {round(xx2,6)} {round(yy2,6)}\n')
                            #print('maketxt')
                      #  cou += 1
                   # print(cou)



            
            
            
            

        
    

In [12]:
copy_path

'/home/ssac26/Downloads/wego/ai_test2_copy'

In [13]:
def file_filtering(path):
    img_1 = getimgdir(path)
    text_1 = gettextdir(path)
    addlist = []
    new_folder = os.path.join(copy_path, os.path.basename(path))
    # os.makedirs(new_folder)
    
   
    
    
    for i in img_1:
        full_fname = os.path.basename(i)
        split_fname = os.path.splitext(full_fname)
        #print(split_fname[0])
        #print(i)
        for j in text_1:
            if split_fname[0] == os.path.splitext(os.path.basename(j))[0]:   # 일치한 파일 827
                
                addlist.append(split_fname[0])
                #print(i)
                #print(j)
                shutil.copy(i, new_folder)
                shutil.move(j, new_folder)
                
    return len(addlist)
    
    
    

In [14]:
#폴더의 모든파일을 읽는 함수
def read_all_file(path):
    file_list = []
    output = os.listdir(path)
    for i in output:
        file_list.append(path+'/'+i)
        
    return file_list
        
        

In [15]:
bbox_folders = os.listdir(basepath)

In [16]:
for bbox_folder in bbox_folders:
    workfolder = os.path.join(basepath, bbox_folder)
    
    print(workfolder)
    parsing(workfolder)
    file_filtering(workfolder)
    workfolder_copy = read_all_file(os.path.join(copy_path, os.path.basename(workfolder)))
    txtcount = 0
    jpgcount = 0
    for i in workfolder_copy:
        if 'txt' in i:
            txtcount +=1
        if 'jpg' in i:
            jpgcount += 1
    print('txt counts:',txtcount)
    print('jpg counts:',jpgcount)
    
    

/home/ssac26/Downloads/wego/ai_test2/Bbox_0892
txt counts: 177
jpg counts: 177
/home/ssac26/Downloads/wego/ai_test2/Bbox_0884
txt counts: 125
jpg counts: 125
/home/ssac26/Downloads/wego/ai_test2/Bbox_0863
txt counts: 138
jpg counts: 138
/home/ssac26/Downloads/wego/ai_test2/Bbox_0882
txt counts: 139
jpg counts: 139
/home/ssac26/Downloads/wego/ai_test2/Bbox_0864
txt counts: 143
jpg counts: 143
/home/ssac26/Downloads/wego/ai_test2/Bbox_0883
txt counts: 143
jpg counts: 143
/home/ssac26/Downloads/wego/ai_test2/Bbox_0887
txt counts: 248
jpg counts: 248
/home/ssac26/Downloads/wego/ai_test2/Bbox_0876
txt counts: 133
jpg counts: 133
/home/ssac26/Downloads/wego/ai_test2/Bbox_0869
txt counts: 144
jpg counts: 144
/home/ssac26/Downloads/wego/ai_test2/Bbox_0865
txt counts: 144
jpg counts: 144
/home/ssac26/Downloads/wego/ai_test2/Bbox_0886
txt counts: 119
jpg counts: 119
/home/ssac26/Downloads/wego/ai_test2/Bbox_0875
txt counts: 139
jpg counts: 139
/home/ssac26/Downloads/wego/ai_test2/Bbox_0870
txt c